In [1]:
import numpy as np
import random
from collections import Counter
import pandas as pd

# Define the sequences and conditions
sequences = [0, 1, 2, 3]
conditions = [0, 1]
nTr = 65 # number of trials
def generate_random_sequence(numbers_list, seq_rep, seed=None):
    # Seed the random number generator if a seed is provided
    if seed is not None:
        random.seed(seed)

    # Create a sequence by sampling from the numbers list
    random_sequence = []
    for i in range(seq_rep):
        random_sequence.append(random.sample(numbers_list, len(numbers_list)))

    return np.concatenate(random_sequence)

# Define the list of numbers
numbers_list = [1, 2, 3, 4]
numbers_list = [0, 1]

# Define the desired length of the sequence
sequence_length = 10

# Generate the random sequence
random_sequence = generate_random_sequence(numbers_list, sequence_length)
print(random_sequence)


[0 1 0 1 1 0 1 0 1 0 0 1 1 0 0 1 0 1 0 1]


In [2]:
## Final solution

import random

def shuffle(array):
    currentIndex = len(array)
    while currentIndex > 0:
        randomIndex = random.randint(0, currentIndex - 1)
        currentIndex -= 1
        array[currentIndex], array[randomIndex] = array[randomIndex], array[currentIndex]
    return array

def generate_trial_states(init_seq):
    seq_cond = np.array([[0,0],[0,1],[1,0],[1,1],[2,0],[2,1],[3,0],[3,1]])
    while True: 
        seq = []
        while len(seq) < 65: 
            seq.clear()
            numbers = list(range(8))
            paths = [shuffle([[a, b] for b in numbers]) for a in numbers]
            if len(seq)==0:
                currNum = init_seq
            else:
                currNum = random.randint(0, 7)
            while True:
                seq.append(currNum)

                if len(paths[currNum - 1]) <= 0:
                    break

                path = paths[currNum - 1].pop()

                if len(paths[currNum - 1]) >= 1 and len(paths[path[1] - 1]) <= 0:
                    continue
                currNum = path[1]
        trial_states = seq_cond[np.array(seq)]

        for t in range(len(trial_states)):
            tS[t] = trial_states[t][0]*2+trial_states[t][1]
        for t in range(len(trial_states)-1):
            trans[t] = tS[t]*8+tS[t+1]
        num_unique_transitions = len(np.unique(trans))
        trans2 = np.zeros(len(seq)-1)
        for t in range(len(seq)-1):
            trans2[t] = seq[t]*8 + seq[t+1]
        # trial_transitions = [(trial_states[i], trial_states[i + 1]) for i in range(len(trial_states) - 1)]
        # # Calculate the number of unique transitions
        # trial_transitions = [tuple(map(tuple, transition)) for transition in trial_transitions]
        # unique_transitions = set(trial_transitions)
        # num_unique_transitions = len(unique_transitions)
        if num_unique_transitions==64:
            break
        
    return trial_states, np.unique(trans), np.unique(trans2)


    # print(paths)


# print(len(seq))
# print(seq)


In [4]:
# trial_states_temp has 65 elements, I need to generate trial_states whose index of the element satisfying index%16==1 should be identical to the element of trial_states_temp
a, b, c = generate_trial_states(3)

NameError: name 'tS' is not defined

In [127]:
def generate_tgt_file_fmri(subj_id, run_id, trial_states):
    # trial_states[k][0]: sequence ID (1,2,3,4), 
    # trial_states[k][1]: condition (Number cue: 0, Spatial visual cue: 1)
    # motor_sequences = [[3, 2, 4, 5, 2, 1, 5, 1, 4],[3, 5, 1, 3, 2, 4, 5, 2, 1], [1, 3, 2, 1, 2, 4, 5, 1, 3], [1, 2, 4, 3, 2, 4, 3, 5, 1]]
    motor_sequences = [[3, 2, 4, 5, 1],[3, 5, 1, 2, 4], [1, 3, 2, 5, 4], [1, 4, 5, 2, 3]]

    M = np.size(motor_sequences,1) # length of motor sequence
    # Set constant values
    # stim_time_value = 15000
    prep_time_value = 1000
    mov_time_lim_value = 3000
    startTime = 8020 # about 8 sec after a triggering signal
    # feedback_value = 1

    # Generate 27 trials
    data = []
    n_horizon = len(motor_sequences[0])
    nTr = len(trial_states)+3 # number of trials = 65, but will add 3 additional trials due to resting periods
    k = 0
    for m in range(nTr):        
        # Create a row of data
        if (m+1)%17 == 0:  # For the last trial of each block consisting of 16 trials, not the first trial
            iti_value = 16000 # 10 s ITI for baseline activity estimation
            row = [startTime, trial_states[k][1]] + motor_sequences[trial_states[k][0]] + [''.join(map(str, motor_sequences[trial_states[k][0]])), iti_value, n_horizon, prep_time_value, mov_time_lim_value]
            k = k+1
        elif (m+1)%17 == 1 and m!=0:# For the last trial of each block consisting of 16 trials, not the first trial
            iti_value = 1000 # 10 s ITI for baseline activity estimation
            row = [startTime, trial_states[k-1][1]] + motor_sequences[trial_states[k-1][0]] + [''.join(map(str, motor_sequences[trial_states[k-1][0]])), iti_value, n_horizon, prep_time_value, mov_time_lim_value]
        else:
            iti_value = 1000 # 1 s ITI for other trials
            row = [startTime, trial_states[k][1]] + motor_sequences[trial_states[k][0]] + [''.join(map(str, motor_sequences[trial_states[k][0]])), iti_value, n_horizon, prep_time_value, mov_time_lim_value]
            k = k+1

        # row = [seq_type, feedback_value] + random_numbers + [''.join(map(str, random_numbers)), iti_value, random_horizons[k], stim_time_value, prep_time_value]

        # Append the row to the data list
        data.append(row)
        startTime = startTime + prep_time_value + mov_time_lim_value + iti_value

    # Create a DataFrame using pandas
    columns = ["startTime", "seqType"] + [f"press{i}" for i in range(1, M+1)] + ["cueP", "iti", "Horizon", "PrepTime", 'MovTimeLim']
    df = pd.DataFrame(data, columns=columns)

    # Save the DataFrame to a CSV file with .tgt extension
    run_id = str(run_id).zfill(2)
    subj_id = str(subj_id).zfill(2)
    filename = f"fmri_ssh{subj_id}_r{run_id}.tgt"
  #  filename = "fmri_ssh00_r00.tgt"
    df.to_csv(filename, sep='\t',  index=False)


In [133]:
seq_cond = [(0,0),(0,1),(1,0),(1,1),(2,0),(2,1),(3,0),(3,1)]
init_seq_conds = np.random.permutation(8)

seq_cond = [(0,0),(0,1),(1,0),(1,1),(2,0),(2,1),(3,0),(3,1)]
matrices = []
transAll = [] 
for s in range(2,30):
    
    # Now the elements of the matrix are the indices of the seq_cond array and I need 3D array of seq_conds using for loop

    init_sequence = np.random.permutation(8)
    for r in range(1,9):
        trial_states, unique_transitions1, unique_transitions2 = generate_trial_states(init_sequence[r-1])
        #print(len(unique_transitions1), len(unique_transitions2))
        if len(unique_transitions1)!=64:
            print('Something wrong')
        if len(unique_transitions2)!=64:
            print('Something wrong')
        generate_tgt_file_fmri(s, r, trial_states)
        for t in range(len(trial_states)):
            tS[t] = trial_states[t][0]*2+trial_states[t][1]
        for t in range(len(trial_states)-1):
            trans[t] = tS[t]*8+tS[t+1]
            transAll.append(len(np.unique(trans)))
            if len(np.unique(trans))!=64:
                print(trans)
            
                
        

In [134]:
transAll

[64,
 64,
 64,
 64,
 64,
 64,
 64,
 64,
 64,
 64,
 64,
 64,
 64,
 64,
 64,
 64,
 64,
 64,
 64,
 64,
 64,
 64,
 64,
 64,
 64,
 64,
 64,
 64,
 64,
 64,
 64,
 64,
 64,
 64,
 64,
 64,
 64,
 64,
 64,
 64,
 64,
 64,
 64,
 64,
 64,
 64,
 64,
 64,
 64,
 64,
 64,
 64,
 64,
 64,
 64,
 64,
 64,
 64,
 64,
 64,
 64,
 64,
 64,
 64,
 64,
 64,
 64,
 64,
 64,
 64,
 64,
 64,
 64,
 64,
 64,
 64,
 64,
 64,
 64,
 64,
 64,
 64,
 64,
 64,
 64,
 64,
 64,
 64,
 64,
 64,
 64,
 64,
 64,
 64,
 64,
 64,
 64,
 64,
 64,
 64,
 64,
 64,
 64,
 64,
 64,
 64,
 64,
 64,
 64,
 64,
 64,
 64,
 64,
 64,
 64,
 64,
 64,
 64,
 64,
 64,
 64,
 64,
 64,
 64,
 64,
 64,
 64,
 64,
 64,
 64,
 64,
 64,
 64,
 64,
 64,
 64,
 64,
 64,
 64,
 64,
 64,
 64,
 64,
 64,
 64,
 64,
 64,
 64,
 64,
 64,
 64,
 64,
 64,
 64,
 64,
 64,
 64,
 64,
 64,
 64,
 64,
 64,
 64,
 64,
 64,
 64,
 64,
 64,
 64,
 64,
 64,
 64,
 64,
 64,
 64,
 64,
 64,
 64,
 64,
 64,
 64,
 64,
 64,
 64,
 64,
 64,
 64,
 64,
 64,
 64,
 64,
 64,
 64,
 64,
 64,
 64,
 64,
 64,
 64,
 64,


In [65]:
tS

array([5., 0., 3., 3., 0., 4., 7., 1., 3., 2., 4., 6., 4., 4., 1., 1., 5.,
       6., 0., 7., 3., 1., 7., 6., 6., 7., 4., 5., 7., 0., 6., 5., 2., 5.,
       1., 4., 3., 6., 2., 6., 1., 0., 2., 2., 7., 7., 5., 5., 4., 0., 0.,
       1., 2., 0., 5., 3., 7., 2., 3., 4., 2., 1., 6., 3., 5.])

In [73]:
trans

array([40.,  3., 27., 24.,  4., 39., 57., 11., 26., 20., 38., 52., 36.,
       33.,  9., 13., 46., 48.,  7., 59., 25., 15., 62., 54., 55., 60.,
       37., 47., 56.,  6., 53., 42., 21., 41., 12., 35., 30., 50., 22.,
       49.,  8.,  2., 18., 23., 63., 61., 45., 44., 32.,  0.,  1., 10.,
       16.,  5., 43., 31., 58., 19., 28., 34., 17., 14., 51., 29.])

In [68]:
tS = np.zeros(len(trial_states))
trans = np.zeros(64)

    

In [8]:
seq_cond = [(0,0),(0,1),(1,0),(1,1),(2,0),(2,1),(3,0),(3,1)]
init_seq_conds = np.random.permutation(8)

nTr = 72
seq_cond = [(0,0),(0,1),(1,0),(1,1),(2,0),(2,1),(3,0),(3,1)]
matrices = []
for s in range(1,3):
    # generate 8 by 8 matrix, every column and row should a randomly permuted array of 0,1,2,3,4,5,6,7
    matrix = np.zeros((8,8),dtype=int)
    row_permutation = np.random.permutation(8)
    col_permutation = np.random.permutation(8)
    for i in range(8):
        matrix[i] = np.roll(row_permutation, i)
    matrix = np.roll(matrix, 1, axis=1)
    matrix = np.roll(matrix, 1, axis=0)
    matrices.append(matrix)
    # Now the elements of the matrix are the indices of the seq_cond array and I need 3D array of seq_conds using for loop
    init_seq_conds = np.zeros((8,8,2),dtype=int)
    for i in range(8):
        for j in range(8):
            init_seq_conds[i,j] = seq_cond[matrix[i,j]]

    for r in range(1,9):
        trial_states = generate_trial_states(init_seq_conds[r-1], nTr)
        generate_tgt_file_fmri(s, r, trial_states)

TypeError: generate_trial_states() takes 1 positional argument but 2 were given

In [ ]:


def generate_trial_states_OLD(init_seq_conds, nTr):
    nBlock = int(nTr/6)-1

    while True:
        # Initialize the list to store trial states
        trial_states = np.zeros((nTr, 2),dtype=int)

        # Set the initial condition for the first trial
        current_sequence = init_seq_conds[0][0]
        current_condition = init_cond  # G=1, V=1
        trial_states[0]= [current_sequence, current_condition]

        # # Generate the trial states
        # # rand_seq = generate_random_sequence(sequences, int(nTr/len(sequences)), seed=42)
        rand_cond = generate_random_sequence(conditions, int(nTr/len(conditions)))
        rand_bin = generate_random_sequence([0, 1], int(nTr/2))
        rep_flag_seq = 1
        rep_flag_cond = 1
        max_rep = 3
        for k in range(nTr-1):  
            # 
            # if rand_bin[k] == 0:
            if ((random.random() < 0.5) and (rep_flag_seq < max_rep)):
                next_sequence = current_sequence
                rep_flag_seq += 1
            else: # randomly choose from the remaining sequences
                # Decide the probability of choosing each sequence depending on its frequency so far, sum of prob_seq should be equal to 1
                other_seqs = [seq for seq in sequences if seq != current_sequence]
                # temp = np.array([np.count_nonzero(trial_states[:k+1,0] == elem) for elem in other_seqs])
                # if any(temp>0):
                #     temp = sum(temp)-temp
                #     prob_seq = temp/sum(temp) 
                # else:
                #     prob_seq = [1/len(other_seqs)]*len(other_seqs)
                # next_sequence = np.random.choice(other_seqs, p = prob_seq)
                next_sequence = np.random.choice(other_seqs)
                rep_flag_seq = 1
            # if rand_cond[k] ==0:
            if ((random.random() < 0.5) and (rep_flag_cond < max_rep)):
                next_condition = current_condition
                rep_flag_cond += 1
            else: # choose the other condition
                next_condition = 1 - current_condition
                rep_flag_cond = 1
            trial_states[k+1] = [next_sequence, next_condition]
            current_sequence = next_sequence
            current_condition = next_condition

        # Calculate frequency of trial transitions from two consecutive trials
        trial_transitions = [(trial_states[i], trial_states[i + 1]) for i in range(len(trial_states) - 1)]

        # Convert NumPy arrays to tuples
        trial_transitions = [tuple(map(tuple, transition)) for transition in trial_transitions]

        # Calculate the number of unique transitions
        unique_transitions = set(trial_transitions)
        num_unique_transitions = len(unique_transitions)

        count_all = []
        for transition, count in Counter(trial_transitions).items():
            count_all.append(count)
        # Define counters for each case
        no_change_count = 0
        sequence_change_count = 0
        condition_change_count = 0
        both_change_count = 0
        # Iterate through trial transitions
        # for transition in trial_transitions:
        #     current_state, next_state = transition
        for blk in range(6): # 6 blocks, each block consisting of 10 trials is defined between 10-s rest periods
            for k in range(nBlock):
                current_state, next_state = trial_states[blk*nBlock+k], trial_states[blk*nBlock+k+1]
                # Check if the sequence and condition remain unchanged
                if current_state[0] == next_state[0] and current_state[1] == next_state[1]:
                    no_change_count += 1
                # Check if only the sequence changes
                elif current_state[0] != next_state[0] and current_state[1] == next_state[1]:
                    sequence_change_count += 1
                # Check if only the condition changes
                elif current_state[0] == next_state[0] and current_state[1] != next_state[1]:
                    condition_change_count += 1
                # Check if both the sequence and condition change
                elif current_state[0] != next_state[0] and current_state[1] != next_state[1]:
                    both_change_count += 1
        
        # If we have found more than 36 unique transitions and largest value in count_all is less than 5, break the loop
        # no_consecutive = has_not_consecutive_same(trial_states_array)
        # no_consecutive_cond = has_not_consecutive_same_condition(trial_states_array)
        # no_consecutive_seq = has_not_consecutive_same_sequence(trial_states_array)
        #if (min(Counter(trial_states[:,1]).values())>0.45*nTr) & (max(Counter(trial_states[:,1]).values())<0.55*nTr) & (max(Counter(trial_states[:,0]).values())<0.3*nTr) & (min(Counter(trial_states[:,0]).values())>0.2*nTr) & (num_unique_transitions > 10) & (max(count_all)<5) & (max(no_change_count, sequence_change_count, condition_change_count, both_change_count) < 0.3*nTr) & (min(no_change_count, sequence_change_count, condition_change_count, both_change_count) > 0.2*nTr):
        if (min(Counter(trial_states[:,1]).values())>0.45*nTr) & (max(Counter(trial_states[:,1]).values())<0.55*nTr) & (max(Counter(trial_states[:,0]).values())<0.3*nTr) & (min(Counter(trial_states[:,0]).values())>0.2*nTr) & (num_unique_transitions > 10) & (max(count_all)<5) & (max(no_change_count, sequence_change_count, condition_change_count, both_change_count) == 15) & (min(no_change_count, sequence_change_count, condition_change_count, both_change_count) == 15):

            break

    # # Print the frequency of trial transitions
    # print("Frequency of Trial Transitions:")
    # for transition, count in Counter(trial_transitions).items():
    #     print(f"Transition: {transition}, Frequency: {count}")
    
    print(f"Number of unique transitions: {num_unique_transitions}")
    print(no_change_count, sequence_change_count, condition_change_count, both_change_count)
    print(f"Number of sequences: {Counter(trial_states[:,0])}")
    print(f"Number of conditions: {Counter(trial_states[:,1])}")
    return trial_states
    # # Define the file name
    # file_name = "trial_states.csv"

    # # Save the trial_states_array as a CSV file
    # np.savetxt(file_name, trial_states_array, delimiter=",", fmt="%s")


In [ ]:
seq_cond = [(0,0),(0,1),(1,0),(1,1),(2,0),(2,1),(3,0),(3,1)]
init_seq_conds = np.random.permutation(seq_cond)
init_seq_conds[:,0]

array([1, 2, 2, 3, 1, 0, 0, 3])

In [ ]:
def generate_trial_states_OLD2(init_seq_conds, nTr):
    nBlock = 8
    nTrBlock = int(nTr/nBlock) # 9 trials per block


    while True:
        # Initialize the list to store trial states
        trial_states = np.zeros((nTr, 2),dtype=int)


        max_rep = 3

        # # Generate the trial states
        for b in range(nBlock):
            rep_flag_seq = 1
            rep_flag_cond = 1
            # Set the initial condition for the first trial

            trial_states[b*nTrBlock]= init_seq_conds[b]
            current_sequence, current_condition = trial_states[b*nTrBlock]

            for k in range(nTrBlock-1):  
                # if rand_bin[k] == 0:
                if ((random.random() < 0.5) and (rep_flag_seq < max_rep)):
                    next_sequence = current_sequence
                    rep_flag_seq += 1
                else: # randomly choose from the remaining sequences
                    # Decide the probability of choosing each sequence depending on its frequency so far, sum of prob_seq should be equal to 1
                    other_seqs = [seq for seq in sequences if seq != current_sequence]
                    # temp = np.array([np.count_nonzero(trial_states[:k+1,0] == elem) for elem in other_seqs])
                    # if any(temp>0):
                    #     temp = sum(temp)-temp
                    #     prob_seq = temp/sum(temp) 
                    # else:
                    #     prob_seq = [1/len(other_seqs)]*len(other_seqs)
                    # next_sequence = np.random.choice(other_seqs, p = prob_seq)
                    next_sequence = np.random.choice(other_seqs)
                    rep_flag_seq = 1
                # if rand_cond[k] ==0:
                if ((random.random() < 0.5) and (rep_flag_cond < max_rep)):
                    next_condition = current_condition
                    rep_flag_cond += 1
                else: # choose the other condition
                    next_condition = 1 - current_condition
                    rep_flag_cond = 1
                trial_states[b*nTrBlock+k+1] = [next_sequence, next_condition]
                current_sequence = next_sequence
                current_condition = next_condition

        # Calculate frequency of trial transitions from two consecutive trials
        trial_transitions = [(trial_states[i], trial_states[i + 1]) for i in range(len(trial_states) - 1)]

        # Convert NumPy arrays to tuples
        trial_transitions = [tuple(map(tuple, transition)) for transition in trial_transitions]

        # Calculate the number of unique transitions
        unique_transitions = set(trial_transitions)
        num_unique_transitions = len(unique_transitions)

        count_all = []
        for transition, count in Counter(trial_transitions).items():
            count_all.append(count)
        # Define counters for each case
        no_change_count = 0
        sequence_change_count = 0
        condition_change_count = 0
        both_change_count = 0
        within_fistfingers_count = 0
        # Iterate through trial transitions
        # for transition in trial_transitions:
        #     current_state, next_state = transition
        for b in range(nBlock): # 6 blocks, each block consisting of 10 trials is defined between 10-s rest periods
            for k in range(nTrBlock-1):
                current_state, next_state = trial_states[b*nTrBlock+k], trial_states[b*nTrBlock+k+1]
                # Check if the sequence and condition remain unchanged
                if current_state[0] == next_state[0] and current_state[1] == next_state[1]:
                    no_change_count += 1
                # Check if only the sequence changes
                elif current_state[0] != next_state[0] and current_state[1] == next_state[1]:
                    sequence_change_count += 1
                # Check if only the condition changes
                elif current_state[0] == next_state[0] and current_state[1] != next_state[1]:
                    condition_change_count += 1
                # Check if both the sequence and condition change
                elif current_state[0] != next_state[0] and current_state[1] != next_state[1]:
                    both_change_count += 1
                if max(current_state[0], next_state[0]) < 2 or min(current_state[0], next_state[0]) >= 2:
                    within_fistfingers_count += 1
        within_fistfingers_count = within_fistfingers_count - (no_change_count+condition_change_count)
        # If we have found more than 36 unique transitions and largest value in count_all is less than 5, break the loop
        # no_consecutive = has_not_consecutive_same(trial_states_array)
        # no_consecutive_cond = has_not_consecutive_same_condition(trial_states_array)
        # no_consecutive_seq = has_not_consecutive_same_sequence(trial_states_array)
        #if (min(Counter(trial_states[:,1]).values())>0.45*nTr) & (max(Counter(trial_states[:,1]).values())<0.55*nTr) & (max(Counter(trial_states[:,0]).values())<0.3*nTr) & (min(Counter(trial_states[:,0]).values())>0.2*nTr) & (num_unique_transitions > 10) & (max(count_all)<5) & (max(no_change_count, sequence_change_count, condition_change_count, both_change_count) < 0.3*nTr) & (min(no_change_count, sequence_change_count, condition_change_count, both_change_count) > 0.2*nTr):

        if ((within_fistfingers_count>=9) & (within_fistfingers_count<=12)) & (min(Counter(trial_states[:,1]).values())>0.45*nTr) & (max(Counter(trial_states[:,1]).values())<0.55*nTr) & (max(Counter(trial_states[:,0]).values())<0.3*nTr) & (min(Counter(trial_states[:,0]).values())>0.2*nTr) & (num_unique_transitions > 10) & (max(count_all)<5) & (max(no_change_count, sequence_change_count, condition_change_count, both_change_count) == 16) & (min(no_change_count, sequence_change_count, condition_change_count, both_change_count) == 16):
            print(f"Number of unique transitions: {num_unique_transitions}")
            print(no_change_count, sequence_change_count, condition_change_count, both_change_count, within_fistfingers_count)
            print(f"Number of sequences: {Counter(trial_states[:,0])}")
            print(f"Number of conditions: {Counter(trial_states[:,1])}")
            break

        # # Print the frequency of trial transitions
        # print("Frequency of Trial Transitions:")
        # for transition, count in Counter(trial_transitions).items():
        #     print(f"Transition: {transition}, Frequency: {count}")
        

    return trial_states
    # # Define the file name
    # file_name = "trial_states.csv"

    # # Save the trial_states_array as a CSV file
    # np.savetxt(file_name, trial_states_array, delimiter=",", fmt="%s")


In [43]:
seq_cond = [(0,0),(0,1),(1,0),(1,1),(2,0),(2,1),(3,0),(3,1)]
init_seq_conds = np.random.permutation(seq_cond)

trial_states = generate_trial_states(init_seq_conds, 72)


Number of unique transitions: 43
16 16 16 16 12
Number of sequences: Counter({1: 21, 0: 19, 3: 17, 2: 15})
Number of conditions: Counter({0: 37, 1: 35})


In [45]:
def generate_tgt_file_fmri(subj_id, run_id, trial_states):
    # trial_states[k][0]: sequence ID (1,2,3,4), 
    # trial_states[k][1]: condition (Number cue: 0, Spatial visual cue: 1)
    motor_sequences = [[3, 2, 4, 5, 2, 1, 5, 1, 4],[3, 5, 1, 3, 2, 4, 5, 2, 1], [1, 3, 2, 1, 2, 4, 5, 1, 3], [1, 2, 4, 3, 2, 4, 3, 5, 1]]
    M = np.size(motor_sequences,1) # length of motor sequence
    # Set constant values
    # stim_time_value = 15000
    prep_time_value = 1000
    mov_time_lim_value = 4000
    # feedback_value = 1

    # Generate 27 trials
    data = []
    n_horizon = 9
    nTr = len(trial_states)
    for k in range(nTr):
        # Create a row of data
        if (k+1)%11 == 0 and k!=0: # Every 11th trial, not the first trial
            iti_value = 10000 # 10 s ITI for baseline activity estimation
        else:
            iti_value = 1000 # 1 s ITI for other trials

        row = [trial_states[k][1]] + motor_sequences[trial_states[k][0]] + [''.join(map(str, motor_sequences[trial_states[k][0]])), iti_value, n_horizon, prep_time_value, mov_time_lim_value]
        # row = [seq_type, feedback_value] + random_numbers + [''.join(map(str, random_numbers)), iti_value, random_horizons[k], stim_time_value, prep_time_value]

        # Append the row to the data list
        data.append(row)

    # Create a DataFrame using pandas
    columns = ["seqType"] + [f"press{i}" for i in range(1, M+1)] + ["cueP", "iti", "Horizon", "PrepTime", 'MovTimeLim']
    df = pd.DataFrame(data, columns=columns)

    # Save the DataFrame to a CSV file with .tgt extension
    run_id = str(run_id).zfill(2)
    subj_id = str(subj_id).zfill(2)
    filename = f"fmri_ssh{subj_id}_r{run_id}.tgt"
    df.to_csv(filename, sep='\t',  index=False)


In [63]:
seq_conds = seq_cond[matrix]

TypeError: only integer scalar arrays can be converted to a scalar index

[[4 2 1 0 3 7 6 5]
 [5 4 2 1 0 3 7 6]
 [6 5 4 2 1 0 3 7]
 [7 6 5 4 2 1 0 3]
 [3 7 6 5 4 2 1 0]
 [0 3 7 6 5 4 2 1]
 [1 0 3 7 6 5 4 2]
 [2 1 0 3 7 6 5 4]]
[[[2 0]
  [1 0]
  [0 1]
  [0 0]
  [1 1]
  [3 1]
  [3 0]
  [2 1]]

 [[2 1]
  [2 0]
  [1 0]
  [0 1]
  [0 0]
  [1 1]
  [3 1]
  [3 0]]

 [[3 0]
  [2 1]
  [2 0]
  [1 0]
  [0 1]
  [0 0]
  [1 1]
  [3 1]]

 [[3 1]
  [3 0]
  [2 1]
  [2 0]
  [1 0]
  [0 1]
  [0 0]
  [1 1]]

 [[1 1]
  [3 1]
  [3 0]
  [2 1]
  [2 0]
  [1 0]
  [0 1]
  [0 0]]

 [[0 0]
  [1 1]
  [3 1]
  [3 0]
  [2 1]
  [2 0]
  [1 0]
  [0 1]]

 [[0 1]
  [0 0]
  [1 1]
  [3 1]
  [3 0]
  [2 1]
  [2 0]
  [1 0]]

 [[1 0]
  [0 1]
  [0 0]
  [1 1]
  [3 1]
  [3 0]
  [2 1]
  [2 0]]]


In [67]:
seq_conds[0,0]

array([2, 0])

Number of unique transitions: 41
16 16 16 16 11
Number of sequences: Counter({0: 21, 2: 20, 3: 16, 1: 15})
Number of conditions: Counter({0: 38, 1: 34})
Number of unique transitions: 41
16 16 16 16 10
Number of sequences: Counter({2: 21, 1: 17, 3: 17, 0: 17})
Number of conditions: Counter({1: 39, 0: 33})
Number of unique transitions: 44
16 16 16 16 10
Number of sequences: Counter({0: 19, 1: 19, 3: 18, 2: 16})
Number of conditions: Counter({0: 38, 1: 34})
Number of unique transitions: 45
16 16 16 16 10
Number of sequences: Counter({0: 21, 2: 19, 1: 17, 3: 15})
Number of conditions: Counter({1: 37, 0: 35})
Number of unique transitions: 42
16 16 16 16 10
Number of sequences: Counter({0: 21, 3: 19, 1: 17, 2: 15})
Number of conditions: Counter({0: 36, 1: 36})
Number of unique transitions: 45
16 16 16 16 9
Number of sequences: Counter({2: 19, 3: 18, 0: 18, 1: 17})
Number of conditions: Counter({0: 37, 1: 35})
Number of unique transitions: 45
16 16 16 16 12
Number of sequences: Counter({2: 20

In [ ]:
trial_states[:4,0]

array([0, 0, 0, 0])

In [ ]:
np.bincount(trial_states[:4,0])[other_seqs]

IndexError: index 1 is out of bounds for axis 0 with size 1

In [ ]:
run_id = 12
formatted_run_id = str(run_id).zfill(2)
formatted_run_id


'12'

In [204]:
sequence_change_count, condition_change_count

(20, 9)

In [203]:
max(sequence_change_count, condition_change_count), min(sequence_change_count, condition_change_count)


(20, 9)

In [201]:
# Define counters for each case
no_change_count = 0
sequence_change_count = 0
condition_change_count = 0
both_change_count = 0

# Iterate through trial transitions
for transition in trial_transitions:
    current_state, next_state = transition

    # Check if the sequence and condition remain unchanged
    if current_state == next_state:
        no_change_count += 1
    # Check if only the sequence changes
    elif current_state[0] != next_state[0] and current_state[1] == next_state[1]:
        sequence_change_count += 1
    # Check if only the condition changes
    elif current_state[1] != next_state[1] and current_state[0] == next_state[0]:
        condition_change_count += 1
    # Check if both the sequence and condition change
    elif current_state[0] != next_state[0] and current_state[1] != next_state[1]:
        both_change_count += 1

# Print the counts
print("Number of cases where:")
print(f"(1) No change from the current to the next trial state: {no_change_count}")
print(f"(2) Only sequence ID changes: {sequence_change_count}")
print(f"(3) Only condition changed: {condition_change_count}")
print(f"(4) Both sequence and condition changed: {both_change_count}")


Number of cases where:
(1) No change from the current to the next trial state: 6
(2) Only sequence ID changes: 20
(3) Only condition changed: 9
(4) Both sequence and condition changed: 29


In [64]:
from collections import Counter
import numpy as np
import random

# Define the sequences and conditions
sequences = [1, 2, 3]
conditions = [0, 1]

while True:
    # Initialize the list to store trial states
    trial_states = []

    # Set the initial condition for the first trial
    current_sequence = random.choice(sequences)
    current_condition = (1, 1)  # G=1, V=1
    trial_states.append((current_sequence, current_condition))

    # Generate the trial states
    for _ in range(72):  # We generate 64 transitions for 65 trials
        next_sequence = random.choice(sequences)
        next_condition = random.choice(conditions)
        trial_states.append((next_sequence, next_condition))

    # Convert trial states to a 2D numpy array
    trial_states_array = np.array(trial_states, dtype=object)

    # Calculate frequency of trial transitions from two consecutive trials
    trial_transitions = [(trial_states_array[i], trial_states_array[i + 1]) for i in range(len(trial_states_array) - 1)]

    # Convert NumPy arrays to tuples
    trial_transitions = [tuple(map(tuple, transition)) for transition in trial_transitions]

    # Calculate the number of unique transitions
    unique_transitions = set(trial_transitions)
    num_unique_transitions = len(unique_transitions)

    count_all = []
    for transition, count in Counter(trial_transitions).items():
        count_all.append(count)
    # If we have found all 36 unique transitions, break the loop
    if num_unique_transitions == 36 & all(np.array(count_all)<4):
        break

# Print the frequency of trial transitions
print("Frequency of Trial Transitions:")
for transition, count in Counter(trial_transitions).items():
    print(f"Transition: {transition}, Frequency: {count}")

print(f"Number of unique transitions: {num_unique_transitions}")

import numpy as np

# Define the file name
file_name = "trial_states.csv"

# Save the trial_states_array as a CSV file
np.savetxt(file_name, trial_states_array, delimiter=",", fmt="%s")



KeyboardInterrupt: 

In [56]:
count_all = []
for transition, count in Counter(trial_transitions).items():
    count_all.append(count)
sum(np.array(count_all)>2)>0

True

In [62]:
all(np.array(count_all)<10)

True

In [14]:
import itertools

# Define the sequences and conditions
sequences = [1, 2, 3]
conditions = [0, 1]

# Generate all possible transitions
all_transitions = list(itertools.product(
    itertools.product(sequences, conditions),
    itertools.product(sequences, conditions)
))

# Print the number of unique transitions
print(f"Number of unique transitions: {len(all_transitions)}")
print("All possible transitions:")
for transition in all_transitions:
    print(transition)


Number of unique transitions: 36
All possible transitions:
((1, 0), (1, 0))
((1, 0), (1, 1))
((1, 0), (2, 0))
((1, 0), (2, 1))
((1, 0), (3, 0))
((1, 0), (3, 1))
((1, 1), (1, 0))
((1, 1), (1, 1))
((1, 1), (2, 0))
((1, 1), (2, 1))
((1, 1), (3, 0))
((1, 1), (3, 1))
((2, 0), (1, 0))
((2, 0), (1, 1))
((2, 0), (2, 0))
((2, 0), (2, 1))
((2, 0), (3, 0))
((2, 0), (3, 1))
((2, 1), (1, 0))
((2, 1), (1, 1))
((2, 1), (2, 0))
((2, 1), (2, 1))
((2, 1), (3, 0))
((2, 1), (3, 1))
((3, 0), (1, 0))
((3, 0), (1, 1))
((3, 0), (2, 0))
((3, 0), (2, 1))
((3, 0), (3, 0))
((3, 0), (3, 1))
((3, 1), (1, 0))
((3, 1), (1, 1))
((3, 1), (2, 0))
((3, 1), (2, 1))
((3, 1), (3, 0))
((3, 1), (3, 1))


In [22]:
transition[0]

(1, 1)

In [27]:
duplicated_transitions = [transition for transition in all_transitions if transition[0][0] == transition[1][0]]
duplicated_transitions = duplicated_transitions[:12]
duplicated_transitions
len(duplicated_transitions)
duplicated_transitions

[((1, 0), (1, 0)),
 ((1, 0), (1, 1)),
 ((1, 1), (1, 0)),
 ((1, 1), (1, 1)),
 ((2, 0), (2, 0)),
 ((2, 0), (2, 1)),
 ((2, 1), (2, 0)),
 ((2, 1), (2, 1)),
 ((3, 0), (3, 0)),
 ((3, 0), (3, 1)),
 ((3, 1), (3, 0)),
 ((3, 1), (3, 1))]

In [16]:
import itertools

# Define the sequences and conditions
sequences = [1, 2, 3]
conditions = [0, 1]

# Generate all possible transitions
all_transitions = list(itertools.product(
    itertools.product(sequences, conditions),
    itertools.product(sequences, conditions)
))

# Duplicate transitions where the sequence ID is repeated
duplicated_transitions = [transition for transition in all_transitions if transition[0][0] == transition[1][0]]
duplicated_transitions = duplicated_transitions[:12]
# Extend the list of transitions
all_transitions.extend(duplicated_transitions)

# Print the number of unique transitions
print(f"Number of unique transitions: {len(set(all_transitions))}")
print("All possible transitions:")
for transition in all_transitions:
    print(transition)
    


Number of unique transitions: 36
All possible transitions:
((1, 0), (1, 0))
((1, 0), (1, 1))
((1, 0), (2, 0))
((1, 0), (2, 1))
((1, 0), (3, 0))
((1, 0), (3, 1))
((1, 1), (1, 0))
((1, 1), (1, 1))
((1, 1), (2, 0))
((1, 1), (2, 1))
((1, 1), (3, 0))
((1, 1), (3, 1))
((2, 0), (1, 0))
((2, 0), (1, 1))
((2, 0), (2, 0))
((2, 0), (2, 1))
((2, 0), (3, 0))
((2, 0), (3, 1))
((2, 1), (1, 0))
((2, 1), (1, 1))
((2, 1), (2, 0))
((2, 1), (2, 1))
((2, 1), (3, 0))
((2, 1), (3, 1))
((3, 0), (1, 0))
((3, 0), (1, 1))
((3, 0), (2, 0))
((3, 0), (2, 1))
((3, 0), (3, 0))
((3, 0), (3, 1))
((3, 1), (1, 0))
((3, 1), (1, 1))
((3, 1), (2, 0))
((3, 1), (2, 1))
((3, 1), (3, 0))
((3, 1), (3, 1))
((1, 0), (1, 0))
((1, 0), (1, 0))
((1, 0), (1, 0))
((1, 0), (1, 0))
((1, 0), (1, 0))
((1, 0), (1, 0))
((1, 1), (1, 1))
((1, 1), (1, 1))
((1, 1), (1, 1))
((1, 1), (1, 1))
((1, 1), (1, 1))
((1, 1), (1, 1))


In [37]:
# Initialize the list to store trial states
trial_states = []

# Generate trial states from the transitions
for i in range(len(all_transitions)):
    if i == 0:
        trial_states.append(all_transitions[i][0])  # Add the first trial state
    trial_states.append(all_transitions[i][1])  # Add the current trial state

# Print the number of trial states
print("Number of trial states:", len(trial_states))

# Print the trial states
print("Trial States:")
for state in trial_states:
    print(state)


Number of trial states: 49
Trial States:
(2, 0)
(2, 0)
(3, 1)
(3, 1)
(2, 1)
(3, 0)
(1, 0)
(3, 1)
(1, 0)
(2, 1)
(1, 1)
(3, 0)
(2, 0)
(1, 1)
(1, 1)
(1, 0)
(2, 0)
(1, 0)
(1, 1)
(1, 0)
(3, 0)
(1, 0)
(1, 1)
(1, 0)
(1, 0)
(1, 1)
(1, 1)
(3, 0)
(1, 1)
(2, 1)
(1, 1)
(1, 1)
(2, 1)
(3, 0)
(1, 0)
(2, 0)
(1, 1)
(3, 0)
(1, 1)
(2, 1)
(3, 1)
(2, 1)
(3, 1)
(3, 1)
(1, 0)
(1, 0)
(2, 0)
(1, 0)
(2, 0)


In [44]:
# Define the sequences and conditions
sequences = [1, 2, 3]
conditions = [0, 1]

# Generate all possible transitions
all_transitions = list(itertools.product(
    itertools.product(sequences, conditions),
    itertools.product(sequences, conditions)
))

# Initialize sorted_transitions with the first transition
sorted_transitions = [all_transitions[0]]

# Repeat the sorting procedure until the number of sorted transitions matches that of all_transitions
while len(sorted_transitions) < len(all_transitions):
    for transition in all_transitions:
        if sorted_transitions[-1][1] == transition[0]:
            sorted_transitions.append(transition)

# Print the number of unique transitions
print(f"Number of unique transitions: {len(set(sorted_transitions))}")
print("All possible transitions:")
for transition in sorted_transitions:
    print(transition)


Number of unique transitions: 3
All possible transitions:
((1, 0), (1, 0))
((1, 0), (1, 0))
((1, 0), (1, 1))
((1, 1), (1, 0))
((1, 0), (1, 0))
((1, 0), (1, 1))
((1, 1), (1, 0))
((1, 0), (1, 0))
((1, 0), (1, 1))
((1, 1), (1, 0))
((1, 0), (1, 0))
((1, 0), (1, 1))
((1, 1), (1, 0))
((1, 0), (1, 0))
((1, 0), (1, 1))
((1, 1), (1, 0))
((1, 0), (1, 0))
((1, 0), (1, 1))
((1, 1), (1, 0))
((1, 0), (1, 0))
((1, 0), (1, 1))
((1, 1), (1, 0))
((1, 0), (1, 0))
((1, 0), (1, 1))
((1, 1), (1, 0))
((1, 0), (1, 0))
((1, 0), (1, 1))
((1, 1), (1, 0))
((1, 0), (1, 0))
((1, 0), (1, 1))
((1, 1), (1, 0))
((1, 0), (1, 0))
((1, 0), (1, 1))
((1, 1), (1, 0))
((1, 0), (1, 0))
((1, 0), (1, 1))
((1, 1), (1, 0))


In [49]:
sorted_transitions

[((1, 0), (1, 0)),
 ((1, 0), (1, 0)),
 ((1, 0), (1, 1)),
 ((1, 1), (1, 0)),
 ((1, 0), (1, 0)),
 ((1, 0), (1, 1)),
 ((1, 1), (1, 0)),
 ((1, 0), (1, 0)),
 ((1, 0), (1, 1)),
 ((1, 1), (1, 0)),
 ((1, 0), (1, 0)),
 ((1, 0), (1, 1)),
 ((1, 1), (1, 0)),
 ((1, 0), (1, 0)),
 ((1, 0), (1, 1)),
 ((1, 1), (1, 0)),
 ((1, 0), (1, 0)),
 ((1, 0), (1, 1)),
 ((1, 1), (1, 0)),
 ((1, 0), (1, 0)),
 ((1, 0), (1, 1)),
 ((1, 1), (1, 0)),
 ((1, 0), (1, 0)),
 ((1, 0), (1, 1)),
 ((1, 1), (1, 0)),
 ((1, 0), (1, 0)),
 ((1, 0), (1, 1)),
 ((1, 1), (1, 0)),
 ((1, 0), (1, 0)),
 ((1, 0), (1, 1)),
 ((1, 1), (1, 0)),
 ((1, 0), (1, 0)),
 ((1, 0), (1, 1)),
 ((1, 1), (1, 0)),
 ((1, 0), (1, 0)),
 ((1, 0), (1, 1)),
 ((1, 1), (1, 0))]

In [48]:
all_transitions

[((1, 0), (1, 0)),
 ((1, 0), (1, 1)),
 ((1, 0), (2, 0)),
 ((1, 0), (2, 1)),
 ((1, 0), (3, 0)),
 ((1, 0), (3, 1)),
 ((1, 1), (1, 0)),
 ((1, 1), (1, 1)),
 ((1, 1), (2, 0)),
 ((1, 1), (2, 1)),
 ((1, 1), (3, 0)),
 ((1, 1), (3, 1)),
 ((2, 0), (1, 0)),
 ((2, 0), (1, 1)),
 ((2, 0), (2, 0)),
 ((2, 0), (2, 1)),
 ((2, 0), (3, 0)),
 ((2, 0), (3, 1)),
 ((2, 1), (1, 0)),
 ((2, 1), (1, 1)),
 ((2, 1), (2, 0)),
 ((2, 1), (2, 1)),
 ((2, 1), (3, 0)),
 ((2, 1), (3, 1)),
 ((3, 0), (1, 0)),
 ((3, 0), (1, 1)),
 ((3, 0), (2, 0)),
 ((3, 0), (2, 1)),
 ((3, 0), (3, 0)),
 ((3, 0), (3, 1)),
 ((3, 1), (1, 0)),
 ((3, 1), (1, 1)),
 ((3, 1), (2, 0)),
 ((3, 1), (2, 1)),
 ((3, 1), (3, 0)),
 ((3, 1), (3, 1))]

In [40]:
# Sort the transitions so that the second element of the (N-1)th transition matches the first element of the Nth transition
sorted_transitions = sorted(all_transitions, key=lambda x: (x[0][0], x[1][0]))

sorted_transitions

[((1, 1), (1, 1)),
 ((1, 1), (1, 1)),
 ((1, 0), (1, 0)),
 ((1, 0), (1, 0)),
 ((1, 1), (1, 1)),
 ((1, 1), (1, 0)),
 ((1, 0), (1, 0)),
 ((1, 0), (1, 0)),
 ((1, 0), (1, 0)),
 ((1, 0), (1, 1)),
 ((1, 1), (1, 1)),
 ((1, 1), (1, 1)),
 ((1, 1), (1, 1)),
 ((1, 0), (1, 0)),
 ((1, 1), (1, 1)),
 ((1, 0), (1, 0)),
 ((1, 1), (2, 1)),
 ((1, 0), (2, 0)),
 ((1, 0), (2, 1)),
 ((1, 1), (2, 0)),
 ((1, 1), (3, 1)),
 ((1, 0), (3, 0)),
 ((1, 1), (3, 0)),
 ((1, 0), (3, 1)),
 ((2, 0), (1, 0)),
 ((2, 1), (1, 1)),
 ((2, 0), (1, 1)),
 ((2, 1), (1, 0)),
 ((2, 0), (2, 0)),
 ((2, 0), (2, 1)),
 ((2, 1), (2, 1)),
 ((2, 1), (2, 0)),
 ((2, 0), (3, 0)),
 ((2, 1), (3, 0)),
 ((2, 0), (3, 1)),
 ((2, 1), (3, 1)),
 ((3, 1), (1, 0)),
 ((3, 1), (1, 1)),
 ((3, 0), (1, 1)),
 ((3, 0), (1, 0)),
 ((3, 0), (2, 1)),
 ((3, 1), (2, 0)),
 ((3, 0), (2, 0)),
 ((3, 1), (2, 1)),
 ((3, 1), (3, 1)),
 ((3, 0), (3, 1)),
 ((3, 1), (3, 0)),
 ((3, 0), (3, 0))]

In [17]:
import itertools
import csv

# Define the sequences and conditions
sequences = [1, 2, 3]
conditions = [0, 1]

# Generate all possible transitions
all_transitions = list(itertools.product(
    itertools.product(sequences, conditions),
    itertools.product(sequences, conditions)
))

# Duplicate transitions where the sequence ID is repeated
duplicated_transitions = [(transition[0], transition[0]) for transition in all_transitions[:12]]

# Extend the list of transitions
all_transitions.extend(duplicated_transitions)

# Save transitions to a CSV file
with open('transitions.csv', 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(['Previous_Sequence', 'Previous_Condition', 'Current_Sequence', 'Current_Condition'])
    for transition in all_transitions:
        writer.writerow([transition[0][0], transition[0][1], transition[1][0], transition[1][1]])

print("Transitions saved to transitions.csv file.")


Transitions saved to transitions.csv file.


In [10]:
from collections import Counter
import numpy as np
import random

# Define the sequences and conditions
sequences = [1, 2, 3, 4]
conditions = [(0, 0), (0, 1), (1, 0), (1, 1)]

while True:
    # Initialize the list to store trial states
    trial_states = []

    # Set the initial condition for the first trial
    current_sequence = random.choice(sequences)
    current_condition = (1, 1)  # G=1, V=1
    trial_states.append((current_sequence, current_condition))

    # Generate the trial states
    for _ in range(64):  # We generate 64 transitions for 65 trials
        next_sequence = random.choice(sequences)
        next_condition = random.choice(conditions)
        trial_states.append((next_sequence, next_condition))

    # Convert trial states to a 2D numpy array
    trial_states_array = np.array(trial_states, dtype=object)

    # Calculate frequency of trial transitions from two consecutive trials
    trial_transitions = [(trial_states_array[i], trial_states_array[i + 1]) for i in range(len(trial_states_array) - 1)]

    # Convert NumPy arrays to tuples
    trial_transitions = [tuple(map(tuple, transition)) for transition in trial_transitions]

    # Calculate the number of unique transitions
    unique_transitions = set(trial_transitions)
    num_unique_transitions = len(unique_transitions)

    # If we have found all 64 unique transitions, break the loop
    if num_unique_transitions == 64:
        break

# Print the frequency of trial transitions
print("Frequency of Trial Transitions:")
for transition, count in Counter(trial_transitions).items():
    print(f"Transition: {transition}, Frequency: {count}")

print(f"Number of unique transitions: {num_unique_transitions}")

import numpy as np

# Define the file name
file_name = "trial_states.csv"

# Save the trial_states_array as a CSV file
np.savetxt(file_name, trial_states_array, delimiter=",", fmt="%s")



Frequency of Trial Transitions:
Transition: ((4, (1, 1)), (1, (1, 0))), Frequency: 1
Transition: ((1, (1, 0)), (3, (0, 0))), Frequency: 1
Transition: ((3, (0, 0)), (2, (1, 0))), Frequency: 1
Transition: ((2, (1, 0)), (4, (1, 1))), Frequency: 1
Transition: ((4, (1, 1)), (4, (0, 1))), Frequency: 1
Transition: ((4, (0, 1)), (1, (0, 1))), Frequency: 1
Transition: ((1, (0, 1)), (4, (1, 0))), Frequency: 1
Transition: ((4, (1, 0)), (2, (1, 0))), Frequency: 1
Transition: ((2, (1, 0)), (1, (1, 1))), Frequency: 1
Transition: ((1, (1, 1)), (3, (0, 0))), Frequency: 1
Transition: ((3, (0, 0)), (4, (1, 0))), Frequency: 1
Transition: ((4, (1, 0)), (4, (0, 1))), Frequency: 1
Transition: ((4, (0, 1)), (4, (1, 1))), Frequency: 1
Transition: ((4, (1, 1)), (3, (0, 1))), Frequency: 1
Transition: ((3, (0, 1)), (4, (1, 0))), Frequency: 1
Transition: ((4, (1, 0)), (1, (0, 0))), Frequency: 1
Transition: ((1, (0, 0)), (3, (0, 0))), Frequency: 1
Transition: ((3, (0, 0)), (3, (1, 1))), Frequency: 1
Transition: ((

In [63]:
import numpy as np
from itertools import permutations, product

# Define the sequences and conditions
sequences = [1, 2, 3, 4]
conditions = [(0, 0), (0, 1), (1, 0), (1, 1)]

# Generate all possible permutations of sequences and conditions
sequence_permutations = permutations(sequences, 2)
condition_permutations = product(conditions, repeat=2)

# Initialize the list to store trial states
trial_states = []

# Generate trial states using permutations
for sequence_permutation, condition_permutation in zip(sequence_permutations, condition_permutations):
    trial_states.append((sequence_permutation[0],) + condition_permutation[0])
    trial_states.append((sequence_permutation[1],) + condition_permutation[1])

# Convert trial states to a structured NumPy array
trial_states_array = np.array(trial_states, dtype=[('sequence', int), ('g', int), ('v', int)])

# Calculate frequency of sequence IDs and transitions
unique_sequences, sequence_counts = np.unique(trial_states_array['sequence'], return_counts=True)
print("Frequency of Sequence IDs:", dict(zip(unique_sequences, sequence_counts)))

unique_transitions, transition_counts = np.unique(trial_states_array[['g', 'v']], axis=0, return_counts=True)
print("Frequency of Transitions:")
for transition, count in zip(unique_transitions, transition_counts):
    print(f"Transition: {transition}, Frequency: {count}")

# Calculate frequency of trial transitions from two consecutive trials
trial_transitions = [(trial_states_array[i], trial_states_array[i + 1]) for i in range(len(trial_states_array) - 1)]

# Flatten the trial transitions
flat_trial_transitions = [tuple(np.concatenate([transition[0], transition[1]])) for transition in trial_transitions]

# Calculate frequency of trial transitions
unique_trial_transitions, trial_transition_counts = np.unique(flat_trial_transitions, return_counts=True)
print("Frequency of Trial Transitions:")
for transition, count in zip(unique_trial_transitions, trial_transition_counts):
    print(f"Transition: {transition}, Frequency: {count}")

# Calculate number of unique transitions
num_unique_transitions = len(unique_trial_transitions)
print(f"Number of unique transitions: {num_unique_transitions}")


Frequency of Sequence IDs: {1: 6, 2: 6, 3: 6, 4: 6}
Frequency of Transitions:
Transition: (0, 0), Frequency: 7
Transition: (0, 1), Frequency: 7
Transition: (1, 0), Frequency: 7
Transition: (1, 1), Frequency: 3


ValueError: zero-dimensional arrays cannot be concatenated